## Noa project

The goal is to convert the Program object into a DreamCoder program, which is expressed via lambda calculus. That way we can run DreamCoder's compression algorithm on the programs we create from our bidirectional search algorithm. You can manually construct the program by composing different classes together from `dreamcoder/program.py`. There should be a way to check that the program you create, when evaluated on the input grids, generates the output grids. These should be written as automatic tests done to make sure that the creations are correct.